# 6.- Test Environment Local

In [ ]:
import os
import docker

In [ ]:
# Create a Docker client
client = docker.from_env()

In [2]:
# Docker image name
docker_image = 'satellite-object-detection:latest'

In [7]:
external_local_path = os.path.abspath('../external')
data_local_path = os.path.abspath('../data')
src_local_path = os.path.abspath('../src')
model_output_local_path = os.path.abspath('../model_output')

In [8]:
# Corrected volume mounting with different local paths
volumes = {
    external_local_path: {'bind': '/app/external', 'mode': 'rw'},
    data_local_path: {'bind': '/app/data', 'mode': 'rw'},
    src_local_path: {'bind': '/app/src', 'mode': 'rw'},
    model_output_local_path: {'bind': '/app/model_output', 'mode': 'rw'},
}

In [ ]:
# Define the command to execute the model training script
command = [
    'python', '/app/src/research/object_detection/model_main_tf2.py',
    '--pipeline_config_path', '/app/src/custom_pipeline_file.config',
    '--model_dir', '/app/model_output',
    '--alsologtostderr',
    '--num_train_steps', '1',
    '--sample_1_of_n_eval_examples', '1'
]

In [ ]:
# Run the container and capture the output
try:
    # Run the container with bash, install libGL, then execute the command
    container = client.containers.run(
        docker_image,         # Docker image name
        command,
        volumes=volumes,      # Volumes to mount
        detach=True,          # Run in background (detach mode)
        stdout=True,          # Enable stdout capture
        stderr=True,          # Enable stderr capture
        tty=False             # Disable TTY, since we don’t need interactivity
    )

    print("Container is running...")

    # Collect and print logs in real-time
    for log in container.logs(stream=True):
        print(log.decode('utf-8'), end='')

    # Wait for the container to finish
    container.wait()

    print("Command executed successfully")

except docker.errors.DockerException as e:
    print(f"Error executing Docker command: {e}")


Container is running...
2025-02-22 23:50:02.731850: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 23:50:02.733810: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 23:50:02.764376: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 23:50:02.764789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-22 23:50:03.657643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.c